In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

# Q3

In [2]:
import os 
os.environ['PYSPARK_PYTHON'] = "/root/anaconda3/bin/python"     # 设置了运行 PySpark 的 Python 解释器的环境变量

spark = SparkSession.builder\
    .config(\
        "spark.jars",  \
        "/shareddata/lab09/kafka-clients-3.5.0.jar, \
        /shareddata/lab09/spark-sql-kafka-0-10_2.12-3.5.0.jar, \
        /shareddata/lab09/spark-token-provider-kafka-0-10_2.12-3.5.0.jar, \
        /shareddata/lab09/commons-pool2-2.12.0.jar") \
    .config("spark.executorEnv.PYSPARK_PYTHON","/root/anaconda3/bin/python")\
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.log.level", "WARN") \
    .appName("ass3_q3").getOrCreate()


24/05/07 10:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Setting Spark log level to "WARN".


## (1)

check the data 

有gender不为0或1的

In [3]:
df = spark.read.csv("data/shopping_data/user_log.csv",header=True)
df.show(5)
df.where((col("gender") != 0) & (col("gender") != 1)).count()

+-------+-------+------+-----------+--------+-----+---+------+---------+------+--------+
|user_id|item_id|cat_id|merchant_id|brand_id|month|day|action|age_range|gender|province|
+-------+-------+------+-----------+--------+-----+---+------+---------+------+--------+
| 328862| 323294|   833|       2882|    2661|   08| 29|     0|        0|     1|  内蒙古|
| 328862| 844400|  1271|       2882|    2661|   08| 29|     0|        1|     1|    山西|
| 328862| 575153|  1271|       2882|    2661|   08| 29|     0|        2|     1|    山西|
| 328862| 996875|  1271|       2882|    2661|   08| 29|     0|        1|     1|  内蒙古|
| 328862|1086186|  1271|       1253|    1049|   08| 29|     0|        0|     2|    浙江|
+-------+-------+------+-----------+--------+-----+---+------+---------+------+--------+
only showing top 5 rows



266736

In [4]:
df = spark.readStream.format("kafka")\
	.option("kafka.bootstrap.servers", "localhost:64050")\
	.option("subscribe", "q3")\
	.option("startingOffsets", "earliest")\
    .load()

In [5]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



时间戳类型是interger，需要转换成datetime类型

.select("key", "data.*")

In [6]:
df1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","CAST (timestamp AS TIMESTAMP)")
schema = "gender INT, action INT"
df2 = df1.selectExpr("key", "from_json(value, '{}') AS data".format(schema), "timestamp")

df_filter = df2.where((col("data.action") == 2) & (col("data.gender") != 0) & (col("data.gender") != 1))
result = df_filter.withWatermark("timestamp", "10 seconds").groupBy("data.gender",window("timestamp", "5 seconds")).agg(count("*").alias("Number of transactions"))
q3_query = result.writeStream.queryName("transaction_count").format("memory").outputMode("update").trigger(processingTime="5 seconds").start()

24/05/07 10:45:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-d434029a-de26-4863-b9ec-3314ced2e7dd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/07 10:45:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [24]:
spark.sql("SELECT window, gender, `Number of transactions` FROM transaction_count").show(20,False)

+------+------+----------------------+
|window|gender|Number of transactions|
+------+------+----------------------+
+------+------+----------------------+



In [8]:
q3_query.stop()